# Kalman filter

## 理論概要

<b>補正後の状態　＝　補正前の状態 ＋ カルマンゲイン ×（本物の観測値－予測された観測値）</b><br><br>
<b>カルマンゲイン ＝ 状態の予測誤差の分散　÷（状態の予測誤差の分散＋観測方程式のノイズの分散）</b>
<br><br>
将来予測が当たる（「状態の予測誤差の分散」が小さい）と判断されていれば、補正は少なくなります。<br>
また、観測方程式におけるノイズが大きいと判断されても、やはり補正幅は小さくなります。<br><br>
<b>今期の状態の予測誤差の分散 ＝ 前期の状態の予測誤差の分散 ＋ 状態方程式のノイズの分散</b>
<br><br>
ただし、これもやはり「本物の観測値」が手に入ると、予測誤差の分散も小さくなります。<br>
「観測値」を基にして「状態の予測誤差の分散」を修正するのも、カルマンフィルタの役割です。<br><br>
<b>補正後の状態の予測誤差の分散 ＝（１ーカルマンゲイン）× 補正前の状態の予測誤差の分散</b>


## ローカルレベルモデルでの実装
<br>
観測方程式<br>
「観測値」　～　「状態」 ＋　ノイズ
<br><br>
状態方程式<br>
「状態」　～　「前期の状態」　＋　ノイズ<br><br>
y : 当期の観測値<br>
x_pre : 前期の状態<br>
p_pre : 前期の状態の予測誤差の分散<br>
sigmaW : 状態方程式のノイズの分散<br>
sigmaV : 観測方程式のノイズの分散<br>
<br>

In [5]:
import pandas as pd
import numpy as np

In [6]:
def local_level_model(y,x_pre,p_pre,sigmaW,sigmaV):
    x_frc = x_pre # ローカルレベルモデルゆえ予測値は前期の値と同じ
    p_frc = p_pre + sigmaW #状態の予測誤差の分散
    k_gain = p_frc / (p_frc + sigmaV) #カルマンゲイン
    x_flt = x_frc + k_gain * (y - x_frc) #カルマンゲインを使って補正された状態
    p_flt = (1 - k_gain) * p_frc # 補正された状態の予測誤差の分散
    return x_flt,p_flt

In [7]:
df = pd.read_csv('../TagGame/Tag_simulation/oni.csv')
df.head(3)

,x,y
0,40.0580,-41.4635
1,37.1892,-42.8135
2,34.6861,-44.1072


In [8]:
#サンプルサイズ
N = len(df.x)
#状態の推定値
x = pd.Series(np.zeros(N))
y = pd.Series(np.zeros(N))
#状態の予測誤差の分散
px = pd.Series(np.zeros(N))
py = pd.Series(np.zeros(N))

#状態の初期値0
x[0] = 0
y[0] = 0
#状態の分散1000
px[0] = 1000
py[0] = 1000

print(x)
print(py)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
1706    0.0
1707    0.0
1708    0.0
1709    0.0
1710    0.0
1711    0.0
1712    0.0
1713    0.0
1714    0.0
1715    0.0
1716    0.0
1717    0.0
1718    0.0
1719    0.0
1720    0.0
1721    0.0
1722    0.0
1723    0.0
1724    0.0
1725    0.0
1726    0.0
1727    0.0
1728    0.0
1729    0.0
1730    0.0
1731    0.0
1732    0.0
1733    0.0
1734    0.0
1735    0.0
Length: 1736, dtype: float64
0       1000.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0

In [10]:
for i in range(0,N):
    x[i+1],px[i+1] = local_level_model(df.x[i],x[i],px[i],sigmaW=1000,sigmaV=10000)
    y[i+1],py[i+1] = local_level_model(df.y[i],y[i],py[i],sigmaW=1000,sigmaV=10000)

predict = pd.DataFrame({
    'x' : x,
    'y' : y
})
predict.to_csv('../TagGame/Tag_simulation/oni_predict.csv')
predict

,x,y
0,0.000000,0.000000
1,6.676333,-6.910583
2,13.100095,-14.469092
3,18.214851,-21.491776
4,21.742293,-27.375796
5,23.816325,-32.127604
6,24.721423,-35.986515
7,24.723662,-39.135077
8,24.045285,-41.664738
9,22.782731,-43.731720
